In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

# Set up the WebDriver (e.g., Chrome)
# Make sure to provide the path to your WebDriver executable.
driver = webdriver.Chrome()

# Navigate to a website
url = 'https://example.com'  # Replace with your target website
driver.get(url)

seats = pd.read_excel('CED_2021_AUST.xlsx')
seats = seats[['CED_CODE_2021', 'CED_NAME_2021', 'STATE_NAME_2021']].drop_duplicates()
seats = seats.rename(columns={'CED_CODE_2021':'quickstats_code'})

seat_codes = seats['quickstats_code'].to_list()

df = pd.DataFrame()

for seat_code in seat_codes:
    try:
        url = 'https://www.abs.gov.au/census/find-census-data/quickstats/2021/CED' + seat_code
        driver.get(url)
        time.sleep(5)
        tables = driver.find_all(By.CSS_SELECTOR, 'div[class_=qsTable]')
        temp = {
            'seat_code': seat_code,
            'indig': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td[2]').text),
            'age': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[2]/div[2]/div[3]/table/tbody/tr[2]/td[1]').text),
            't_ed': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td[2]').text),
            'income': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[5]/div[2]/div[9]/table/tbody/tr[2]/td[1]').text[1:].replace(',', '')),
            'Ancestry_table': pd.read_html(str(tables[7].find('table')))
        }
        df = pd.concat(
            [df, pd.DataFrame(temp, index=[0])], ignore_index=True
        )
    except:
        pass

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

# Set up the WebDriver (e.g., Chrome)
# Make sure to provide the path to your WebDriver executable.
driver = webdriver.Chrome()

seats = pd.read_excel('CED_2021_AUST.xlsx')
seats = seats[['CED_CODE_2021', 'CED_NAME_2021', 'STATE_NAME_2021']].drop_duplicates()
seats = seats.rename(columns={'CED_CODE_2021':'quickstats_code'})

seat_codes = seats['quickstats_code'].to_list()

driver.get('https://www.abs.gov.au/census/find-census-data/quickstats/2021/CED' + seat_codes[0])

In [5]:
tables = driver.find_elements(By.XPATH, '/html/body/div[1]/main/div[3]/div/div[2]/div[3]/div[1]/div[4]/div[2]/div[1]/table')
pd.read_html(str(tables[7].find('table')))

AttributeError: 'WebElement' object has no attribute 'find'